<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [1]:
!pip install pandas
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 145.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 179.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 141.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 149.1 MB/s eta 0:00:00


In [2]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 146.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 161.8 MB/s eta 0:00:0000:01


## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [4]:
duplicates = df.drop(columns=['ResponseId']).duplicated()
num_duplicates = duplicates.sum()
print("Number of duplicate rows (excluding 'ResponseId'):", num_duplicates)
df_cleaned=df[~duplicates]

Number of duplicate rows (excluding 'ResponseId'): 487


In [5]:
# Write your code here
print(df_cleaned.info())

# Count of missing values per column
missing_counts = df_cleaned.isnull().sum()
print("\nMissing values per column:\n", missing_counts)

<class 'pandas.core.frame.DataFrame'>
Index: 64950 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 57.0+ MB
None

Missing values per column:
 ResponseId                 0
MainBranch                 0
Age                        0
Employment                 0
RemoteWork             10546
                       ...  
JobSatPoints_11        35505
SurveyLength            8768
SurveyEase              8712
ConvertedCompYearly    41515
JobSat                 35824
Length: 114, dtype: int64


<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [6]:
# Write your code here
print(df_cleaned.describe())

         ResponseId      CompTotal       WorkExp  JobSatPoints_1  \
count  64950.000000   3.374000e+04  29658.000000    29324.000000   
mean   32511.068822  2.963841e+145     11.466957       18.581094   
std    18802.882779  5.444117e+147      9.168709       25.966221   
min        1.000000   0.000000e+00      0.000000        0.000000   
25%    16238.250000   6.000000e+04      4.000000        0.000000   
50%    32475.500000   1.100000e+05      9.000000       10.000000   
75%    48733.750000   2.500000e+05     16.000000       22.000000   
max    65437.000000  1.000000e+150     50.000000      100.000000   

       JobSatPoints_4  JobSatPoints_5  JobSatPoints_6  JobSatPoints_7  \
count    29393.000000    29411.000000    29450.000000     29448.00000   
mean         7.522140       10.060857       24.343232        22.96522   
std         18.422661       21.833836       27.089360        27.01774   
min          0.000000        0.000000        0.000000         0.00000   
25%          0.000000 

### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [7]:
# Write your code here
print(df_cleaned['EdLevel'].value_counts(dropna=False))

EdLevel
Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                          24942
Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                                       15557
Some college/university study without earning a degree                                 7651
Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     5793
NaN                                                                                    4166
Professional degree (JD, MD, Ph.D, Ed.D, etc.)                                         2970
Associate degree (A.A., A.S., etc.)                                                    1793
Primary/elementary school                                                              1146
Something else                                                                          932
Name: count, dtype: int64


In [8]:
country= df_cleaned['Country'].value_counts(dropna=False)
#country.tail(50)
country.head(60)
#print(df_cleaned['Country'].value_counts(dropna=False))

Country
United States of America                                11095
NaN                                                      6020
Germany                                                  4947
India                                                    4231
United Kingdom of Great Britain and Northern Ireland     3224
Ukraine                                                  2672
France                                                   2110
Canada                                                   2104
Poland                                                   1534
Netherlands                                              1449
Brazil                                                   1375
Italy                                                    1341
Australia                                                1260
Spain                                                    1123
Sweden                                                   1016
Russian Federation                                        925


In [9]:
country.tail(60)

Country
Malawi                                   12
Benin                                    11
Jamaica                                  11
Turkmenistan                             11
Bahrain                                  11
Mongolia                                 10
Senegal                                   9
Togo                                      9
Somalia                                   9
Trinidad and Tobago                       8
Oman                                      8
Congo, Republic of the...                 8
Kuwait                                    8
Sudan                                     7
Mozambique                                7
Tajikistan                                7
Barbados                                  6
Mauritania                                5
Botswana                                  5
Cape Verde                                5
Suriname                                  5
Antigua and Barbuda                       5
Namibia                 

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [10]:
## Write your code here
               
edlevel_mapping = {
    "Bachelor’s degree (B.A., B.S., B.Eng., etc.)": "Bachelor’s degree",
    "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)": "Master’s degree",
    "Professional degree (JD, MD, Ph.D, Ed.D, etc.)": "Professional degree",
    "Associate degree (A.A., A.S., etc.)": "Associate degree",
    "Some college/university study without earning a degree": "College/University without degree",
    "Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)": "Secondary school",
    "Primary/elementary school": "Primary school",
    "Something else": "Other"
}
df_cleaned.loc[:,'EdLevel'] = df_cleaned['EdLevel'].str.strip().map(edlevel_mapping)
df_cleaned.loc[:,'EdLevel']=df_cleaned['EdLevel'].fillna('Unknown')

In [11]:
print(df_cleaned['EdLevel'].value_counts(dropna=False))

EdLevel
Bachelor’s degree                    24942
Master’s degree                      15557
College/University without degree     7651
Secondary school                      5793
Unknown                               4166
Professional degree                   2970
Associate degree                      1793
Primary school                        1146
Other                                  932
Name: count, dtype: int64


In [54]:
country_mapping = {
    "Congo, Republic of the...": "Congo",
    "Democratic Republic of the Congo": "Congo",
    "Lao People's Democratic Republic": "Lao",
    "Libyan Arab Jamahiriya": "Libyan",
    "Democratic People's Republic of Korea": "Korea",
    "Micronesia, Federated States of...": "Micronesia",
    "Democratic People's Republic of Korea": "Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "Russian Federation":"Russian",
    "Czech Republic":"Czech",
    "Iran, Islamic Republic of...":"Iran",
    "Hong Kong (S.A.R.)":"Hong Kong",
    "Viet Nam":"VietNam"
}
#df_cleaned.loc[:,'Country'] = df_cleaned['Country'].str.strip().map(country_mapping)
df_cleaned.loc[:,'Country'] = df_cleaned['Country'].str.strip()

#Replace only matching entries, keep others
df_cleaned.loc[:, 'Country'] = df_cleaned['Country'].replace(country_mapping)

print(df_cleaned['Country'].value_counts(dropna=False))

Country
United States      11095
NaN                 6020
Germany             4947
India               4231
United Kingdom      3224
                   ...  
Micronesia             1
Nauru                  1
Chad                   1
Djibouti               1
Solomon Islands        1
Name: count, Length: 185, dtype: int64


In [12]:
country= df_cleaned['Country'].value_counts(dropna=False)
country.tail(60)

Country
Malawi                                   12
Benin                                    11
Jamaica                                  11
Turkmenistan                             11
Bahrain                                  11
Mongolia                                 10
Senegal                                   9
Togo                                      9
Somalia                                   9
Trinidad and Tobago                       8
Oman                                      8
Congo, Republic of the...                 8
Kuwait                                    8
Sudan                                     7
Mozambique                                7
Tajikistan                                7
Barbados                                  6
Mauritania                                5
Botswana                                  5
Cape Verde                                5
Suriname                                  5
Antigua and Barbuda                       5
Namibia                 

### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [15]:
## Write your code here

df_cleaned.loc[:, 'Employment'] = df_cleaned['Employment'].fillna("")

#Split the values in 'Employment' column into lists
if 'Employment_split' not in df_cleaned.columns:
    df_cleaned.loc[:, 'Employment'] = df_cleaned['Employment'].fillna("")
    df_cleaned.loc[:, 'Employment_split'] = df_cleaned['Employment'].apply(lambda x: [val.strip() for val in x.split(';')])

    from sklearn.preprocessing import MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    employment_dummies = pd.DataFrame(mlb.fit_transform(df_cleaned['Employment_split']),
                                       columns=mlb.classes_,
                                       index=df_cleaned.index)

    # Only add new columns if they don't already exist
    df_cleaned = pd.concat([df_cleaned, employment_dummies], axis=1)

In [16]:
df_cleaned.iloc[:,-20:-1 ].head()

,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,ConvertedCompYearly,JobSat,Employment_split,"Employed, full-time","Employed, part-time",I prefer not to say,"Independent contractor, freelancer, or self-employed","Not employed, and not looking for work","Not employed, but looking for work",Retired,"Student, full-time"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Employed, full-time]",1,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[Employed, full-time]",1,0,0,0,0,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN,NaN,"[Employed, full-time]",1,0,0,0,0,0,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN,NaN,"[Student, full-time]",0,0,0,0,0,0,0,1
4,NaN,NaN,NaN,NaN,NaN,NaN,Too short,Easy,NaN,NaN,"[Student, full-time]",0,0,0,0,0,0,0,1


### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [17]:
## Write your code here
missing_counts = df_cleaned.isnull().sum().sort_values(ascending=False)
print("Columns with the most missing values:")
print(missing_counts[missing_counts > 0])

Columns with the most missing values:
AINextMuch less integrated    63802
AINextLess integrated         62595
AINextNo change               52452
AINextMuch more integrated    51512
EmbeddedAdmired               48217
                              ...  
LanguageHaveWorkedWith         5205
YearsCode                      5081
NEWSOSites                     4669
LearnCode                      4462
AISelect                       4051
Length: 108, dtype: int64


<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [20]:
## Write your code here
mean_value = df_cleaned['ConvertedCompYearly'].mean()
df_cleaned.fillna({'ConvertedCompYearly':mean_value}, inplace=True)

<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [29]:
## Write your code here
mostFrequent_RemoteWork=df_cleaned['RemoteWork'].mode()[0]
df_cleaned.fillna({'RemoteWork':mostFrequent_RemoteWork}, inplace=True)


### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [32]:
## Write your code here
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_cleaned['ConvertedCompYearly_MinMax'] = scaler.fit_transform(df_cleaned[['ConvertedCompYearly']])

<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [33]:
## Write your code here
import numpy as np

# Avoid errors from log(0) by filtering positive values only
df_cleaned = df_cleaned[df_cleaned['ConvertedCompYearly'] > 0]
df_cleaned['ConvertedCompYearly_Log'] = np.log(df_cleaned['ConvertedCompYearly'])

### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [34]:
df_cleaned['YearsCodePro']

0        NaN
1         17
2         27
3        NaN
4        NaN
        ... 
65432      3
65433    NaN
65434      5
65435      2
65436    NaN
Name: YearsCodePro, Length: 64950, dtype: object

In [35]:
## Write your code here
# First, convert 'YearsCodePro' to numeric
df_cleaned['YearsCodePro'] = pd.to_numeric(df_cleaned['YearsCodePro'], errors='coerce')

# Define experience levels
def experience_level(years):
    if pd.isna(years):
        return 'Unknown'
    elif years < 3:
        return 'Beginner'
    elif years < 8:
        return 'Intermediate'
    else:
        return 'Advanced'

# Apply the function
df_cleaned['ExperienceLevel'] = df_cleaned['YearsCodePro'].apply(experience_level)

# Check the result
print(df_cleaned['ExperienceLevel'].value_counts())

ExperienceLevel
Advanced        25703
Unknown         16246
Intermediate    16194
Beginner         6807
Name: count, dtype: int64


### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
